<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_05_1_langchain_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 5: LangChain: Data Extraction**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 5 Material

* **Part 5.1: Structured Output Parser** [[Video]](https://www.youtube.com/watch?v=62CSR141VRE) [[Notebook]](t81_559_class_05_1_langchain_data.ipynb)
* Part 5.2: Other Parsers (CSV, JSON, Pandas, Datetime) [[Video]](https://www.youtube.com/watch?v=VXm8gPzU3qc) [[Notebook]](t81_559_class_05_2_parsers.ipynb)
* Part 5.3: Pydantic parser [[Video]](https://www.youtube.com/watch?v=dc4fn-W60hg) [[Notebook]](t81_559_class_05_3_pydantic.ipynb)
* Part 5.4: Custom Output Parser [[Video]](https://www.youtube.com/watch?v=jBpkAblQC_U) [[Notebook]](t81_559_class_05_4_custom_parsers.ipynb)
* Part 5.5: Output-Fixing Parser [[Video]](https://www.youtube.com/watch?v=_txWiLjf4bo) [[Notebook]](t81_559_class_05_5_output_fixing_parsers.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: not using Google CoLab


# 5.1: Structured Output Parser

LangChain offers a diverse array of output parsers designed to efficiently extract and structure information from the outputs returned by large language models (LLMs). These output parsers play a crucial role in LangChain's architecture, typically forming integral components of what are known as LangChain chains. These chains are configurable sequences of operations that process and interact with model outputs to perform complex tasks. To facilitate the creation and management of these chains, LangChain introduces the LangChain Expression Language (LCEL).

Next, we will delve into how LCEL enables the advanced creation and execution of LangChain chains, and subsequently explore how different output parsers can be utilized within these chains to achieve precise and actionable insights from LLM outputs.

## LangChain Expression Language, or LCEL

LangChain Expression Language, or [LCEL](https://python.langchain.com/docs/expression_language/), provides a declarative way to easily compose chains. From its inception, LCEL has supported transitioning prototypes to production without any code changes. This includes everything from simple "prompt + LLM" chains to highly complex chains that some users have successfully implemented with hundreds of steps in production environments. Here are several reasons why you might choose LCEL:

* [First-class streaming support](https://python.langchain.com/docs/expression_language/streaming/): By building your chains with LCEL, you achieve the optimal time-to-first-token, which is the time elapsed until the first output chunk appears. In some instances, this means streaming tokens directly from an LLM to a streaming output parser, delivering parsed, incremental chunks of output at the same rate the LLM provider releases the raw tokens.

* [Async support](https://python.langchain.com/docs/expression_language/interface/): Chains built with LCEL can use both synchronous APIs (for example, in your Jupyter notebook during prototyping) and asynchronous APIs (for example, in a LangServe server). This dual capability allows the same code to perform efficiently in both prototypes and production, handling many concurrent requests on the same server.

* [Optimized parallel execution](https://python.langchain.com/docs/expression_language/primitives/parallel/): LCEL automatically executes parallelizable steps in your chains simultaneously, whether you are using synchronous or asynchronous interfaces, minimizing latency.

* [Retries and fallbacks](https://python.langchain.com/docs/guides/productionization/fallbacks/): You can configure retries and fallbacks for any part of your LCEL chain, enhancing reliability at scale. We are also enhancing streaming support for retries and fallbacks to improve reliability without increasing latency.

* [Access to intermediate results](https://python.langchain.com/docs/expression_language/interface/#async-stream-events-beta): For more complex chains, accessing intermediate results can be crucial. This feature allows end-users to see progress or helps developers debug the chain. Intermediate results are streamable and available on every LangServe server.

* [Input and output schemas](https://python.langchain.com/docs/expression_language/interface/#input-schema): Every LCEL chain comes with Pydantic and JSONSchema schemas inferred from your chain's structure. These schemas are essential for validating inputs and outputs and are a core feature of LangServe.

* [Seamless LangSmith tracing](https://python.langchain.com/docs/langsmith/): As chains grow in complexity, it's vital to trace each step. LCEL automatically logs all steps to LangSmith for enhanced observability and debuggability.

* [Seamless LangServe deployment](https://python.langchain.com/docs/langserve/): Deploying a chain created with LCEL is straightforward using LangServe, facilitating seamless transitions from development to production.

# Introduction to the StructuredOutputParser

LangChain offers a variety of tools designed to help process and extract information from the output of large language models (LLMs). In this section, we will explore the capabilities of the Structured Output Parser, which is particularly useful when you need to return information across multiple fields. This parser is adept at organizing and segmenting model output into distinct categories, making the data more manageable and interpretable. Although the Pydantic/JSON parser provides a more robust and feature-rich option for handling complex data structures, the Structured Output Parser is an excellent choice for environments where computational resources are limited, or when using less powerful models. It offers a straightforward and efficient way to structure output without overwhelming the model or the system.

We begin by creating a ResponseSchema for each value we wish to extract. We must describe each value. We construct the StructuredOutputParser from a list of these schemas.

In [2]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(
        name="source",
        description="source used to answer the user's question, should be a website.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

We now construct a prompt template that allows both the question and formatting instructions to be specified. The schemas that we just created will generate the formatting instructions.

In [3]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

Before we query the LLM, let's provide a question and see how LangChang constructs the prompt.

In [4]:
question = "When was the Python programming language introduced?"

print(prompt.invoke(question).text)

answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // answer to the user's question
	"source": string  // source used to answer the user's question, should be a website.
}
```
When was the Python programming language introduced?


As you can see, the schemas LangChain uses the schemas to specify a JSON format to return the answer. With the answer in this JSON format, it is straightforward to parse the individual values.

We now construct a chain to use the StructuredOutputParser and query the LLM.

In [5]:
MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)
chain = prompt | llm | output_parser

Now, we present a question, an answer, and a source for the question.

In [6]:
result = chain.invoke({"question": question})
print(result)

{'answer': 'Python programming language was introduced in 1991.', 'source': 'https://en.wikipedia.org/wiki/Python_(programming_language)'}


## Detect and Translate

We will now try an example with more values. The following program accepts text in any language and will translate it into French, Spanish, and Chinese.

In [7]:
response_schemas = [
    ResponseSchema(name="detected", description="The language of the user's input"),
    ResponseSchema(name="spanish", description="Spanish translation"),
    ResponseSchema(name="french", description="French translation"),
    ResponseSchema(name="chinese", description="Chinese translation"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="translate into the requested languages.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser

We begin by trying an English sentence and observe it translated into the other three languages.

In [8]:
question = "When was the Python programming language introduced?"
result = chain.invoke({"question": question})
print(result)

{'detected': 'en', 'spanish': '¿Cuándo se introdujo el lenguaje de programación Python?', 'french': 'Quand le langage de programmation Python a-t-il été introduit?', 'chinese': 'Python编程语言是什么时候推出的？'}


We also observe that it can detect and translate Swahili.

In [9]:
question = "Sijui nini kinaendelea?"
result = chain.invoke({"question": question})
print(result)

{'detected': 'Swahili', 'spanish': '¿Qué está pasando?', 'french': "Qu'est-ce qui se passe?", 'chinese': '我不知道发生了什么？'}


# Module 5 Assignment

You can find the first assignment here: [assignment 5](https://github.com/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_class5.ipynb)